# Workload rtapp + Idle States Residency Analysis

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup(level=100)

# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
%matplotlib inline

import os

# Support to access the remote target
from env import TestEnv

# Support to access cpuidle information from the target
from devlib import *

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Periodic

# Support for trace events analysis
from trace import Trace
from trace import ResidencyTime, ResidencyData

# DataFrame support
import pandas as pd
from pandas import DataFrame

# Trappy (plots) support
from trappy import ILinePlot
from trappy.stats.grammar import Parser

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# CPUIdle break residency statistics

In [3]:
def dfCPUIdleStateEstimationStatistics(ia, cpu, residency):
    
    #residency = residency / 1000000
    
    idle_df = ia._dfg_trace_event('cpu_idle')
    cpu_idle = idle_df[idle_df.cpu_id == cpu]

    # Generate valid idle states
    available_idles = sorted(idle_df.state.unique())
    available_idles = available_idles[1:]

    cpu_is_idle = ia._trace.getCPUActiveSignal(cpu) ^ 1
    cpu_idle = cpu_idle.join(cpu_is_idle.to_frame(name='is_idle'),
                                 how='outer')
    cpu_idle.fillna(method='ffill', inplace=True)

    idle_time = pd.DataFrame({'over_est' : 0, 'under_est' : 0, 'good_est' : 0, 'cpu' : 0}, index=available_idles)

    for i in available_idles:
        idle_state = cpu_idle.state.apply(
            lambda x: 1 if x == i else 0
        )
        #print idle_state
        idle_t = cpu_idle.is_idle * idle_state
        #print idle_t
        # Compute total time by integrating the square wave
        #print ia._trace.integrate_square_wave(idle_t)
        idle_t.iloc[-1] = 0.0

        # Compact signal to obtain only 1-0-1-0 sequences
        comp_sig = idle_t.loc[idle_t.shift() != idle_t]
        #print comp_sig
        if comp_sig.iloc[0] == 0.0:
            idle_interval = (comp_sig.iloc[2::2].index - comp_sig.iloc[1:-1:2].index)
        else:
            idle_interval = (comp_sig.iloc[1::2].index - comp_sig.iloc[:-1:2].index)
        
        idle_interval *= 1000000
        
        idle_time.at[i, 'cpu']  = cpu
        idle_time.at[i, 'over_est']  = len(idle_interval[idle_interval < residency[i]])
        idle_time.at[i, 'under_est'] = len(idle_interval[idle_interval > residency[i+1]])
        idle_time.at[i, 'good_est']  = len(idle_interval[(idle_interval >= residency[i]) & (idle_interval < residency[i+1])])          
        
    return idle_time

In [4]:
from collections import namedtuple

ResidencyTimes = namedtuple('ResidencyTimes', ['label', 'stats'])

def dfAllCPUIdleStateEstimationStatistics(ia, residency, cpus=None):

    if not ia._trace.hasEvents('cpu_idle'):
        ia._log.warning('Events [cpu_idle] not found, '
                        'plot DISABLED!')
        return

    if cpus is None:
        # Generate plots only for available CPUs
        cpuidle_data = ia._dfg_trace_event('cpu_idle')
        _cpus = range(cpuidle_data.cpu_id.max() + 1)
    else:
        _cpus = listify(cpus)

    # Generate valid idle states
    idle_df = ia._dfg_trace_event('cpu_idle')
    available_idles = sorted(idle_df.state.unique())
    available_idles = available_idles[1:]
    
    residencies = pd.DataFrame()
    total_r = pd.DataFrame({'over_est' : 0, 'under_est' : 0, 'good_est' : 0, 'cpu' : -1}, index=available_idles)
    for cpu in _cpus:
        r = dfCPUIdleStateEstimationStatistics(ia, cpu, residency)
        residencies = pd.concat([residencies, r])
        total_r['over_est']  = total_r['over_est']  + r['over_est']
        total_r['under_est'] = total_r['under_est'] + r['under_est']
        total_r['good_est']  = total_r['good_est']  + r['good_est']
        
    residencies = pd.concat([residencies, total_r])
    return residencies

# Hikey620 Platform Configurations

In [5]:
# Setup target configuration
hikey620_conf = {
    "platform"     : 'linux',
    "board"        : 'hikey',
    "host"         : "192.168.1.20",
    "username"     : "root",
    "password"     : "root",
    "results_dir"  : "cpuidle_post_opt",
    "modules"      : [ 'cpufreq', 'cpuidle' ],
    "ftrace"       : {
        "events" : [ "cpu_idle", "cpu_capacity", "cpu_frequency", "sched_switch", "sched_load_avg_cpu", "cpuidle_interval_stats", "cpuidle_state_idx",
                     "cpuidle_factor", "irq_handler_entry", "ipi_entry_rcuidle", "softirq_raise", "sched_migrate_task" ],
        "buffsize" : 50 * 1024,
    },
    "tools"        : [ 'trace-cmd', 'taskset', 'rt-app', 'sysbench' ],
    "rtapp-calib"  : {
        "0": 465, "1": 465, "2": 465, "3": 465, "4": 465, "5": 465, "6": 465, "7": 465,
    },    
}

In [6]:
hikey620_platform = {
    'kernel': {
        'major': 4,
        'sha1': '78e8b90',
        'version': '57 SMP PREEMPT Sat Jul 1 15:42:00 CST 2017',
        'version_number': 4,
        'parts': [4, 4, 74],
        'rc': None,
        'release': '4.4.74-g78e8b90',
        'minor': 74
    },
    
    'freqs': {
        'big':    [ 280000, 432000, 729000, 960000, 1200000 ],
        'little': [ 280000, 432000, 729000, 960000, 1200000 ]
    },
    
    'nrg_model': {
        'big': {
            'cluster': {
                'nrg_max': 112
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 607
            }
        },
        'little': {
            'cluster': {
                'nrg_max': 112
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 607
            }
        }
    },
    
    'clusters': {
        'big': [4, 5, 6, 7],
        'little': [0, 1, 2, 3]
    },
    'cpus_count': 8,
    'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]
}

In [7]:
# Support to access the remote target
from env import TestEnv

te = TestEnv(hikey620_conf)
target = te.target

target.cpufreq.set_all_governors('userspace')
target.cpufreq.set_frequency(0, 1200000)

rfreq = target.cpufreq.get_frequency(0)
print "cpu frequency:", rfreq

2018-07-26 20:14:47,987 INFO    : TestEnv      : Using base path: /home/leoy/Work2/Develop/tools/lisa
2018-07-26 20:14:47,992 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-07-26 20:14:47,993 WARNING : TestEnv      : Wipe previous contents of the results folder:
2018-07-26 20:14:48,001 WARNING : TestEnv      :    /home/leoy/Work2/Develop/tools/lisa/results/cpuidle_post_opt
2018-07-26 20:14:48,010 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cpufreq']
2018-07-26 20:14:48,012 INFO    : TestEnv      : Connecting linux target:
2018-07-26 20:14:48,013 INFO    : TestEnv      :   username : root
2018-07-26 20:14:48,015 INFO    : TestEnv      :       host : 192.168.1.20
2018-07-26 20:14:48,016 INFO    : TestEnv      :   password : root
2018-07-26 20:14:48,018 INFO    : TestEnv      : Connection settings:
2018-07-26 20:14:48,019 INFO    : TestEnv      :    {'username': 'root', 'host': '192.168.1.20', 'password': 'root'}
2018-07-26 20:14:48,029 DEBUG 

2018-07-26 20:14:58,274 DEBUG   : ssh          : (cat /sys/devices/system/cpu/cpu0/cpufreq/scaling_available_frequencies); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:14:58,438 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:14:58,603 DEBUG   : ssh          : (echo 1200000 > '/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:14:58,766 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq'); __devlib_ec=$?; echo; echo $__devlib_ec


cpu frequency: 1200000


# Workload execution utility

In [8]:
def execute(te, wload, res_dir):
    
    logging.info('# Create results folder for this execution')
    !mkdir {res_dir}
    
    logging.info('# Setup FTrace')
    te.ftrace.start()

    #logging.info('## Start energy sampling')
    #te.emeter.reset()

    logging.info('### Start RTApp execution')
    wload.run(out_dir=res_dir)

    #logging.info('## Read energy consumption: %s/energy.json', res_dir)
    #nrg_report = te.emeter.report(out_dir=res_dir)

    logging.info('# Stop FTrace')
    te.ftrace.stop()

    trace_file = os.path.join(res_dir, 'trace.dat')
    logging.info('# Save FTrace: %s', trace_file)
    te.ftrace.get_trace(trace_file)

    logging.info('# Save platform description: %s/platform.json', res_dir)
    plt, plt_file = te.platform_dump(res_dir)
    
    logging.info('# Report collected data:')
    logging.info('   %s', res_dir)
    !ls -la {res_dir}
    
    return plt, plt_file, trace_file

# Single task RTApp workload (period: 10ms, dc: 1%)

In [9]:
rtapp_name = 'example_10ms_1pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p1': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 1,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:14:58,878 INFO    : Workload     : Setup new workload example_10ms_1pct
2018-07-26 20:14:58,980 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:14:59,042 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:14:59,044 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:14:59,046 INFO    : Workload     : ------------------------
2018-07-26 20:14:59,048 INFO    : Workload     : task [task_p1], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:14:59,050 INFO    : Workload     :  | loops count: 1
2018-07-26 20:14:59,053 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:14:59,055 INFO    : Workload     : |  period    10000 [us], duty_cycle   1 %
2018-07-26 20:14:59,056 INFO    : Workload     : |  run_time    100 [us], sleep_time   9900 [us]
2018-07-26 20:14:59,059 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_1pct_0

total 7356
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:15 .
drwxrwxr-x 3 leoy leoy    4096 Jul 26 20:14 ..
-rw-r--r-- 1 leoy leoy     586 Jul 26 20:15 example_10ms_1pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:15 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:15 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p10-0.log
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:15 rt-app-task_p1-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p15-0.log
-rw-rw-r-- 1 leoy leoy 6885376 Jul 26 20:15 trace.dat


,cpu,good_est,over_est,under_est
0,0,58,0,4
1,0,9,38,32
2,0,92,86,0
0,1,29,0,10
1,1,16,12,25
2,1,352,41,0
0,2,13,0,10
1,2,11,13,21
2,2,854,25,0
0,3,37,0,9


# Single task RTApp workload (period: 5ms, dc: 1%)

In [10]:
rtapp_name = 'example_5ms_1pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 1,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:15:20,357 INFO    : Workload     : Setup new workload example_5ms_1pct
2018-07-26 20:15:20,459 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:15:20,522 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:15:20,525 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:15:20,526 INFO    : Workload     : ------------------------
2018-07-26 20:15:20,528 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:15:20,530 INFO    : Workload     :  | loops count: 1
2018-07-26 20:15:20,532 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:15:20,533 INFO    : Workload     : |  period     5000 [us], duty_cycle   1 %
2018-07-26 20:15:20,535 INFO    : Workload     : |  run_time     50 [us], sleep_time   4950 [us]
2018-07-26 20:15:20,536 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_1pct_00.

total 7796
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:15 .
drwxrwxr-x 4 leoy leoy    4096 Jul 26 20:15 ..
-rw-r--r-- 1 leoy leoy     584 Jul 26 20:15 example_5ms_1pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:15 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:15 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p30-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p3-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p35-0.log
-rw-rw-r-- 1 leoy leoy 7213056 Jul 26 20:15 trace.dat


,cpu,good_est,over_est,under_est
0,0,72,0,9
1,0,12,36,23
2,0,84,95,0
0,1,35,0,5
1,1,17,18,30
2,1,2054,47,0
0,2,18,0,4
1,2,11,5,17
2,2,84,20,0
0,3,27,0,7


# Single task RTApp workload (period: 10ms, dc: 3%)

In [11]:
rtapp_name = 'example_10ms_3pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 3,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:15:41,655 INFO    : Workload     : Setup new workload example_10ms_3pct
2018-07-26 20:15:41,758 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:15:41,820 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:15:41,822 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:15:41,824 INFO    : Workload     : ------------------------
2018-07-26 20:15:41,825 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:15:41,826 INFO    : Workload     :  | loops count: 1
2018-07-26 20:15:41,828 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:15:41,829 INFO    : Workload     : |  period    10000 [us], duty_cycle   3 %
2018-07-26 20:15:41,831 INFO    : Workload     : |  run_time    300 [us], sleep_time   9700 [us]
2018-07-26 20:15:41,833 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_3pct_0

total 7440
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:16 .
drwxrwxr-x 5 leoy leoy    4096 Jul 26 20:15 ..
-rw-r--r-- 1 leoy leoy     586 Jul 26 20:15 example_10ms_3pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:15 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:16 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p30-0.log
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:15 rt-app-task_p3-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:15 rt-app-task_p35-0.log
-rw-rw-r-- 1 leoy leoy 6971392 Jul 26 20:16 trace.dat


,cpu,good_est,over_est,under_est
0,0,56,0,6
1,0,16,31,25
2,0,313,79,0
0,1,36,0,10
1,1,26,28,35
2,1,44,45,0
0,2,28,0,10
1,2,15,27,32
2,2,68,17,0
0,3,12,0,3


# Single task RTApp workload (period: 5ms, dc: 3%)

In [12]:
rtapp_name = 'example_5ms_3pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 3,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:16:04,315 INFO    : Workload     : Setup new workload example_5ms_3pct
2018-07-26 20:16:04,422 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:16:04,485 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:16:04,487 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:16:04,491 INFO    : Workload     : ------------------------
2018-07-26 20:16:04,492 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:16:04,494 INFO    : Workload     :  | loops count: 1
2018-07-26 20:16:04,495 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:16:04,497 INFO    : Workload     : |  period     5000 [us], duty_cycle   3 %
2018-07-26 20:16:04,498 INFO    : Workload     : |  run_time    150 [us], sleep_time   4850 [us]
2018-07-26 20:16:04,500 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_3pct_00.

total 7824
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:16 .
drwxrwxr-x 6 leoy leoy    4096 Jul 26 20:16 ..
-rw-r--r-- 1 leoy leoy     585 Jul 26 20:16 example_5ms_3pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:16 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:16 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:16 rt-app-task_p30-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:16 rt-app-task_p3-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:16 rt-app-task_p35-0.log
-rw-rw-r-- 1 leoy leoy 7241728 Jul 26 20:16 trace.dat


,cpu,good_est,over_est,under_est
0,0,15,0,7
1,0,12,12,22
2,0,113,416,0
0,1,27,0,9
1,1,20,18,25
2,1,116,27,0
0,2,6,0,2
1,2,8,2,10
2,2,2092,33,0
0,3,17,0,2


# Single task RTApp workload (period: 10ms, dc: 5%)

In [13]:
rtapp_name = 'example_10ms_5pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p5': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 5,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:16:25,573 INFO    : Workload     : Setup new workload example_10ms_5pct
2018-07-26 20:16:25,680 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:16:25,743 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:16:25,744 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:16:25,747 INFO    : Workload     : ------------------------
2018-07-26 20:16:25,750 INFO    : Workload     : task [task_p5], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:16:25,752 INFO    : Workload     :  | loops count: 1
2018-07-26 20:16:25,753 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:16:25,754 INFO    : Workload     : |  period    10000 [us], duty_cycle   5 %
2018-07-26 20:16:25,756 INFO    : Workload     : |  run_time    500 [us], sleep_time   9500 [us]
2018-07-26 20:16:25,757 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_5pct_0

total 6824
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:16 .
drwxrwxr-x 7 leoy leoy    4096 Jul 26 20:16 ..
-rw-r--r-- 1 leoy leoy     586 Jul 26 20:16 example_10ms_5pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:16 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:16 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:16 rt-app-task_p5-0.log
-rw-rw-r-- 1 leoy leoy 6840320 Jul 26 20:16 trace.dat


,cpu,good_est,over_est,under_est
0,0,30,0,10
1,0,18,35,23
2,0,124,113,0
0,1,18,0,5
1,1,13,9,13
2,1,1104,16,0
0,2,10,0,3
1,2,4,8,13
2,2,80,14,0
0,3,10,0,2


# Single task RTApp workload (period: 5ms, dc: 5%)

In [14]:
rtapp_name = 'example_5ms_5pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p5': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 5,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:16:46,967 INFO    : Workload     : Setup new workload example_5ms_5pct
2018-07-26 20:16:47,069 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:16:47,132 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:16:47,134 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:16:47,137 INFO    : Workload     : ------------------------
2018-07-26 20:16:47,139 INFO    : Workload     : task [task_p5], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:16:47,141 INFO    : Workload     :  | loops count: 1
2018-07-26 20:16:47,144 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:16:47,145 INFO    : Workload     : |  period     5000 [us], duty_cycle   5 %
2018-07-26 20:16:47,147 INFO    : Workload     : |  run_time    250 [us], sleep_time   4750 [us]
2018-07-26 20:16:47,149 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_5pct_00.

total 7240
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:17 .
drwxrwxr-x 8 leoy leoy    4096 Jul 26 20:16 ..
-rw-r--r-- 1 leoy leoy     585 Jul 26 20:17 example_5ms_5pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:17 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:17 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:17 rt-app-task_p5-0.log
-rw-rw-r-- 1 leoy leoy 7143424 Jul 26 20:17 trace.dat


,cpu,good_est,over_est,under_est
0,0,128,0,9
1,0,9,10,16
2,0,117,76,0
0,1,11,0,5
1,1,12,8,12
2,1,105,44,0
0,2,7,0,6
1,2,9,11,19
2,2,80,23,0
0,3,10,0,0


# Single task RTApp workload (period: 10ms, dc: 10%)

In [15]:
rtapp_name = 'example_10ms_10pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p10': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 10, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:17:07,978 INFO    : Workload     : Setup new workload example_10ms_10pct
2018-07-26 20:17:08,081 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:17:08,144 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:17:08,146 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:17:08,149 INFO    : Workload     : ------------------------
2018-07-26 20:17:08,151 INFO    : Workload     : task [task_p10], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:17:08,154 INFO    : Workload     :  | loops count: 1
2018-07-26 20:17:08,156 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:17:08,158 INFO    : Workload     : |  period    10000 [us], duty_cycle  10 %
2018-07-26 20:17:08,160 INFO    : Workload     : |  run_time   1000 [us], sleep_time   9000 [us]
2018-07-26 20:17:08,161 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_10pc

total 6860
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:17 .
drwxrwxr-x 9 leoy leoy    4096 Jul 26 20:17 ..
-rw-r--r-- 1 leoy leoy     589 Jul 26 20:17 example_10ms_10pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:17 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:17 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:17 rt-app-task_p10-0.log
-rw-rw-r-- 1 leoy leoy 6877184 Jul 26 20:17 trace.dat


,cpu,good_est,over_est,under_est
0,0,7,0,2
1,0,9,3,9
2,0,529,119,0
0,1,11,0,3
1,1,13,3,7
2,1,153,31,0
0,2,5,0,3
1,2,8,4,5
2,2,883,20,0
0,3,13,0,2


# Single task RTApp workload (period: 5ms, dc: 10%)

In [16]:
rtapp_name = 'example_5ms_10pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p10': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 10, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:17:30,338 INFO    : Workload     : Setup new workload example_5ms_10pct
2018-07-26 20:17:30,441 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:17:30,504 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:17:30,506 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:17:30,509 INFO    : Workload     : ------------------------
2018-07-26 20:17:30,510 INFO    : Workload     : task [task_p10], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:17:30,513 INFO    : Workload     :  | loops count: 1
2018-07-26 20:17:30,515 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:17:30,517 INFO    : Workload     : |  period     5000 [us], duty_cycle  10 %
2018-07-26 20:17:30,518 INFO    : Workload     : |  run_time    500 [us], sleep_time   4500 [us]
2018-07-26 20:17:30,520 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_10pct_

total 7292
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:17 .
drwxrwxr-x 10 leoy leoy    4096 Jul 26 20:17 ..
-rw-r--r--  1 leoy leoy     587 Jul 26 20:17 example_5ms_10pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:17 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:17 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:17 rt-app-task_p10-0.log
-rw-rw-r--  1 leoy leoy 7196672 Jul 26 20:17 trace.dat


,cpu,good_est,over_est,under_est
0,0,67,0,4
1,0,13,13,14
2,0,117,193,0
0,1,5,0,2
1,1,7,4,6
2,1,2086,20,0
0,2,6,0,5
1,2,11,6,11
2,2,55,10,0
0,3,10,0,2


# Single task RTApp workload (period: 10ms, dc: 15%)

In [17]:
rtapp_name = 'example_10ms_15pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p15': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 15, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:17:52,358 INFO    : Workload     : Setup new workload example_10ms_15pct
2018-07-26 20:17:52,461 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:17:52,525 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:17:52,527 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:17:52,529 INFO    : Workload     : ------------------------
2018-07-26 20:17:52,531 INFO    : Workload     : task [task_p15], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:17:52,533 INFO    : Workload     :  | loops count: 1
2018-07-26 20:17:52,534 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:17:52,536 INFO    : Workload     : |  period    10000 [us], duty_cycle  15 %
2018-07-26 20:17:52,538 INFO    : Workload     : |  run_time   1500 [us], sleep_time   8500 [us]
2018-07-26 20:17:52,540 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_15pc

total 6740
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:18 .
drwxrwxr-x 11 leoy leoy    4096 Jul 26 20:17 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:18 example_10ms_15pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:18 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:18 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:18 rt-app-task_p15-0.log
-rw-rw-r--  1 leoy leoy 6754304 Jul 26 20:18 trace.dat


,cpu,good_est,over_est,under_est
0,0,15,0,4
1,0,5,6,11
2,0,105,129,0
0,1,2,0,2
1,1,7,2,7
2,1,76,20,0
0,2,7,0,3
1,2,14,3,13
2,2,1058,20,0
0,3,4,0,1


# Single task RTApp workload (period: 5ms, dc: 15%)

In [18]:
rtapp_name = 'example_5ms_15pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p15': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 15, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:18:13,017 INFO    : Workload     : Setup new workload example_5ms_15pct
2018-07-26 20:18:13,119 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:18:13,182 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:18:13,183 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:18:13,185 INFO    : Workload     : ------------------------
2018-07-26 20:18:13,186 INFO    : Workload     : task [task_p15], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:18:13,189 INFO    : Workload     :  | loops count: 1
2018-07-26 20:18:13,191 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:18:13,193 INFO    : Workload     : |  period     5000 [us], duty_cycle  15 %
2018-07-26 20:18:13,195 INFO    : Workload     : |  run_time    750 [us], sleep_time   4250 [us]
2018-07-26 20:18:13,197 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_15pct_

total 7188
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:18 .
drwxrwxr-x 12 leoy leoy    4096 Jul 26 20:18 ..
-rw-r--r--  1 leoy leoy     587 Jul 26 20:18 example_5ms_15pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:18 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:18 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:18 rt-app-task_p15-0.log
-rw-rw-r--  1 leoy leoy 7090176 Jul 26 20:18 trace.dat


,cpu,good_est,over_est,under_est
0,0,12,0,5
1,0,4,14,24
2,0,102,139,0
0,1,14,0,6
1,1,9,4,23
2,1,140,37,0
0,2,11,0,5
1,2,11,4,12
2,2,1852,19,0
0,3,0,0,4


# Single task RTApp workload (period: 10ms, dc: 20%)

In [19]:
rtapp_name = 'example_10ms_20pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 20, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:18:36,114 INFO    : Workload     : Setup new workload example_10ms_20pct
2018-07-26 20:18:36,216 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:18:36,280 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:18:36,282 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:18:36,286 INFO    : Workload     : ------------------------
2018-07-26 20:18:36,288 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:18:36,290 INFO    : Workload     :  | loops count: 1
2018-07-26 20:18:36,292 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:18:36,293 INFO    : Workload     : |  period    10000 [us], duty_cycle  20 %
2018-07-26 20:18:36,295 INFO    : Workload     : |  run_time   2000 [us], sleep_time   8000 [us]
2018-07-26 20:18:36,297 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_20pc

total 6888
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:18 .
drwxrwxr-x 13 leoy leoy    4096 Jul 26 20:18 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:18 example_10ms_20pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:18 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:18 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:18 rt-app-task_p20-0.log
-rw-rw-r--  1 leoy leoy 6905856 Jul 26 20:18 trace.dat


,cpu,good_est,over_est,under_est
0,0,24,0,2
1,0,12,10,17
2,0,363,89,0
0,1,5,0,2
1,1,13,5,15
2,1,152,39,0
0,2,10,0,6
1,2,9,4,12
2,2,1090,20,0
0,3,6,0,1


# Single task RTApp workload (period: 5ms, dc: 20%)

In [20]:
rtapp_name = 'example_5ms_20pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 20, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:18:56,886 INFO    : Workload     : Setup new workload example_5ms_20pct
2018-07-26 20:18:56,989 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:18:57,051 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:18:57,054 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:18:57,056 INFO    : Workload     : ------------------------
2018-07-26 20:18:57,058 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:18:57,060 INFO    : Workload     :  | loops count: 1
2018-07-26 20:18:57,062 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:18:57,064 INFO    : Workload     : |  period     5000 [us], duty_cycle  20 %
2018-07-26 20:18:57,065 INFO    : Workload     : |  run_time   1000 [us], sleep_time   4000 [us]
2018-07-26 20:18:57,067 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_20pct_

total 7272
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:19 .
drwxrwxr-x 14 leoy leoy    4096 Jul 26 20:18 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:19 example_5ms_20pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:19 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:19 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:19 rt-app-task_p20-0.log
-rw-rw-r--  1 leoy leoy 7176192 Jul 26 20:19 trace.dat


,cpu,good_est,over_est,under_est
0,0,25,0,1
1,0,17,10,10
2,0,736,102,0
0,1,8,0,3
1,1,13,2,5
2,1,134,26,0
0,2,10,0,0
1,2,8,13,18
2,2,103,24,0
0,3,14,0,7


# Single task RTApp workload (period: 10ms, dc: 25%)

In [21]:
rtapp_name = 'example_10ms_25pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p25': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 25, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:19:17,882 INFO    : Workload     : Setup new workload example_10ms_25pct
2018-07-26 20:19:17,984 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:19:18,047 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:19:18,049 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:19:18,050 INFO    : Workload     : ------------------------
2018-07-26 20:19:18,053 INFO    : Workload     : task [task_p25], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:19:18,055 INFO    : Workload     :  | loops count: 1
2018-07-26 20:19:18,058 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:19:18,059 INFO    : Workload     : |  period    10000 [us], duty_cycle  25 %
2018-07-26 20:19:18,061 INFO    : Workload     : |  run_time   2500 [us], sleep_time   7500 [us]
2018-07-26 20:19:18,063 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_25pc

total 6884
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:19 .
drwxrwxr-x 15 leoy leoy    4096 Jul 26 20:19 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:19 example_10ms_25pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:19 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:19 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:19 rt-app-task_p25-0.log
-rw-rw-r--  1 leoy leoy 6901760 Jul 26 20:19 trace.dat


,cpu,good_est,over_est,under_est
0,0,73,0,6
1,0,10,21,19
2,0,338,132,0
0,1,7,0,7
1,1,7,15,23
2,1,109,28,0
0,2,6,0,3
1,2,4,1,2
2,2,78,29,0
0,3,10,0,0


# Single task RTApp workload (period: 5ms, dc: 25%)

In [22]:
rtapp_name = 'example_5ms_25pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p25': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 25, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:19:39,299 INFO    : Workload     : Setup new workload example_5ms_25pct
2018-07-26 20:19:39,401 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:19:39,464 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:19:39,466 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:19:39,467 INFO    : Workload     : ------------------------
2018-07-26 20:19:39,470 INFO    : Workload     : task [task_p25], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:19:39,472 INFO    : Workload     :  | loops count: 1
2018-07-26 20:19:39,474 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:19:39,476 INFO    : Workload     : |  period     5000 [us], duty_cycle  25 %
2018-07-26 20:19:39,478 INFO    : Workload     : |  run_time   1250 [us], sleep_time   3750 [us]
2018-07-26 20:19:39,480 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_25pct_

total 7260
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:19 .
drwxrwxr-x 16 leoy leoy    4096 Jul 26 20:19 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:19 example_5ms_25pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:19 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:19 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:19 rt-app-task_p25-0.log
-rw-rw-r--  1 leoy leoy 7163904 Jul 26 20:19 trace.dat


,cpu,good_est,over_est,under_est
0,0,71,0,9
1,0,13,41,27
2,0,165,128,0
0,1,18,0,5
1,1,17,16,21
2,1,2036,20,0
0,2,24,0,10
1,2,11,21,32
2,2,61,25,0
0,3,15,0,2


# Single task RTApp workload (period: 10ms, dc: 30%)

In [23]:
rtapp_name = 'example_10ms_30pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p30': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 30, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:20:00,259 INFO    : Workload     : Setup new workload example_10ms_30pct
2018-07-26 20:20:00,366 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:20:00,429 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:20:00,430 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:20:00,433 INFO    : Workload     : ------------------------
2018-07-26 20:20:00,435 INFO    : Workload     : task [task_p30], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:20:00,437 INFO    : Workload     :  | loops count: 1
2018-07-26 20:20:00,439 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:20:00,441 INFO    : Workload     : |  period    10000 [us], duty_cycle  30 %
2018-07-26 20:20:00,442 INFO    : Workload     : |  run_time   3000 [us], sleep_time   7000 [us]
2018-07-26 20:20:00,444 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_30pc

total 6884
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:20 .
drwxrwxr-x 17 leoy leoy    4096 Jul 26 20:20 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:20 example_10ms_30pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:20 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:20 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:20 rt-app-task_p30-0.log
-rw-rw-r--  1 leoy leoy 6901760 Jul 26 20:20 trace.dat


,cpu,good_est,over_est,under_est
0,0,19,0,9
1,0,16,27,26
2,0,331,151,0
0,1,17,0,8
1,1,10,25,29
2,1,477,18,0
0,2,19,0,2
1,2,12,13,18
2,2,651,27,0
0,3,22,0,7


# Single task RTApp workload (period: 5ms, dc: 30%)

In [24]:
rtapp_name = 'example_5ms_30pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p30': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 30, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:20:21,193 INFO    : Workload     : Setup new workload example_5ms_30pct
2018-07-26 20:20:21,295 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:20:21,358 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:20:21,360 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:20:21,361 INFO    : Workload     : ------------------------
2018-07-26 20:20:21,363 INFO    : Workload     : task [task_p30], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:20:21,365 INFO    : Workload     :  | loops count: 1
2018-07-26 20:20:21,367 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:20:21,370 INFO    : Workload     : |  period     5000 [us], duty_cycle  30 %
2018-07-26 20:20:21,372 INFO    : Workload     : |  run_time   1500 [us], sleep_time   3500 [us]
2018-07-26 20:20:21,375 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_30pct_

total 7300
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:20 .
drwxrwxr-x 18 leoy leoy    4096 Jul 26 20:20 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:20 example_5ms_30pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:20 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:20 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:20 rt-app-task_p30-0.log
-rw-rw-r--  1 leoy leoy 7204864 Jul 26 20:20 trace.dat


,cpu,good_est,over_est,under_est
0,0,43,0,4
1,0,11,10,12
2,0,196,137,0
0,1,14,0,2
1,1,21,7,28
2,1,2091,16,0
0,2,14,0,2
1,2,4,17,21
2,2,60,5,0
0,3,8,0,0


# Single task RTApp workload (period: 10ms, dc: 35%)

In [25]:
rtapp_name = 'example_10ms_35pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p35': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 35, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:20:43,484 INFO    : Workload     : Setup new workload example_10ms_35pct
2018-07-26 20:20:43,586 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:20:43,649 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:20:43,651 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:20:43,653 INFO    : Workload     : ------------------------
2018-07-26 20:20:43,656 INFO    : Workload     : task [task_p35], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:20:43,657 INFO    : Workload     :  | loops count: 1
2018-07-26 20:20:43,660 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:20:43,661 INFO    : Workload     : |  period    10000 [us], duty_cycle  35 %
2018-07-26 20:20:43,663 INFO    : Workload     : |  run_time   3500 [us], sleep_time   6500 [us]
2018-07-26 20:20:43,665 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_35pc

total 6836
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:21 .
drwxrwxr-x 19 leoy leoy    4096 Jul 26 20:20 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:21 example_10ms_35pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:21 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:21 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:20 rt-app-task_p35-0.log
-rw-rw-r--  1 leoy leoy 6852608 Jul 26 20:21 trace.dat


,cpu,good_est,over_est,under_est
0,0,29,0,6
1,0,8,15,22
2,0,305,83,0
0,1,25,0,3
1,1,11,27,42
2,1,86,31,0
0,2,24,0,4
1,2,9,15,19
2,2,66,25,0
0,3,29,0,8


# Single task RTApp workload (period: 5ms, dc: 35%)

In [26]:
rtapp_name = 'example_5ms_35pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p35': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 35, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:21:05,417 INFO    : Workload     : Setup new workload example_5ms_35pct
2018-07-26 20:21:05,521 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:21:05,583 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:21:05,585 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:21:05,586 INFO    : Workload     : ------------------------
2018-07-26 20:21:05,588 INFO    : Workload     : task [task_p35], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:21:05,589 INFO    : Workload     :  | loops count: 1
2018-07-26 20:21:05,591 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:21:05,592 INFO    : Workload     : |  period     5000 [us], duty_cycle  35 %
2018-07-26 20:21:05,593 INFO    : Workload     : |  run_time   1750 [us], sleep_time   3250 [us]
2018-07-26 20:21:05,595 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_35pct_

total 7408
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:21 .
drwxrwxr-x 20 leoy leoy    4096 Jul 26 20:21 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:21 example_5ms_35pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:21 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:21 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:21 rt-app-task_p35-0.log
-rw-rw-r--  1 leoy leoy 7315456 Jul 26 20:21 trace.dat


,cpu,good_est,over_est,under_est
0,0,136,0,9
1,0,7,20,32
2,0,189,143,0
0,1,25,0,7
1,1,12,30,31
2,1,44,25,0
0,2,303,0,7
1,2,316,12,398
2,2,1396,24,0
0,3,18,0,7


# Single task RTApp workload (period: 10ms, dc: 40%)

In [27]:
rtapp_name = 'example_10ms_40pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p40': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 40, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:21:26,606 INFO    : Workload     : Setup new workload example_10ms_40pct
2018-07-26 20:21:26,712 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:21:26,774 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:21:26,776 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:21:26,778 INFO    : Workload     : ------------------------
2018-07-26 20:21:26,780 INFO    : Workload     : task [task_p40], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:21:26,782 INFO    : Workload     :  | loops count: 1
2018-07-26 20:21:26,783 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:21:26,785 INFO    : Workload     : |  period    10000 [us], duty_cycle  40 %
2018-07-26 20:21:26,786 INFO    : Workload     : |  run_time   4000 [us], sleep_time   6000 [us]
2018-07-26 20:21:26,788 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_40pc

total 6880
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:21 .
drwxrwxr-x 21 leoy leoy    4096 Jul 26 20:21 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:21 example_10ms_40pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:21 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:21 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:21 rt-app-task_p40-0.log
-rw-rw-r--  1 leoy leoy 6897664 Jul 26 20:21 trace.dat


,cpu,good_est,over_est,under_est
0,0,20,0,9
1,0,13,15,15
2,0,1103,84,0
0,1,14,0,6
1,1,14,5,17
2,1,340,40,0
0,2,8,0,6
1,2,11,16,17
2,2,104,30,0
0,3,21,0,5


# Single task RTApp workload (period: 5ms, dc: 40%)

In [28]:
rtapp_name = 'example_5ms_40pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p40': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 40, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:21:47,891 INFO    : Workload     : Setup new workload example_5ms_40pct
2018-07-26 20:21:47,994 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:21:48,057 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:21:48,058 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:21:48,060 INFO    : Workload     : ------------------------
2018-07-26 20:21:48,063 INFO    : Workload     : task [task_p40], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:21:48,065 INFO    : Workload     :  | loops count: 1
2018-07-26 20:21:48,067 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:21:48,069 INFO    : Workload     : |  period     5000 [us], duty_cycle  40 %
2018-07-26 20:21:48,071 INFO    : Workload     : |  run_time   2000 [us], sleep_time   3000 [us]
2018-07-26 20:21:48,073 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_40pct_

total 7540
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:22 .
drwxrwxr-x 22 leoy leoy    4096 Jul 26 20:21 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:22 example_5ms_40pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:22 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:22 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:22 rt-app-task_p40-0.log
-rw-rw-r--  1 leoy leoy 7450624 Jul 26 20:22 trace.dat


,cpu,good_est,over_est,under_est
0,0,21,0,2
1,0,9,11,15
2,0,182,125,0
0,1,633,0,3
1,1,1165,1,10
2,1,1054,23,0
0,2,10,0,3
1,2,7,5,18
2,2,80,26,0
0,3,10,0,7
